## Predict next letter
'Hello world'

In [1]:
import torch
import torch.nn as nn

## Sample text data

In [2]:
text = "hello world"
chars = sorted(list(set(text)))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = {i: c for i, c in enumerate(chars)}
vocab_size = len(chars)
print(idx2char)

{0: ' ', 1: 'd', 2: 'e', 3: 'h', 4: 'l', 5: 'o', 6: 'r', 7: 'w'}


## Prepare data

In [3]:
seq_length = 4
X = []
y = []
for i in range(len(text) - seq_length):
    input_seq = [char2idx[c] for c in text[i:i + seq_length]]
    target = char2idx[text[i + seq_length]]
    X.append(input_seq)
    y.append(target)
print('X=', X)
print('Y=', y)

X= [[3, 2, 4, 4], [2, 4, 4, 5], [4, 4, 5, 0], [4, 5, 0, 7], [5, 0, 7, 5], [0, 7, 5, 6], [7, 5, 6, 4]]
Y= [5, 0, 7, 5, 6, 4, 1]


## Convert to torch tensor

In [4]:
X = torch.tensor(X, dtype=torch.long)
y = torch.tensor(y, dtype=torch.long)
X

tensor([[3, 2, 4, 4],
        [2, 4, 4, 5],
        [4, 4, 5, 0],
        [4, 5, 0, 7],
        [5, 0, 7, 5],
        [0, 7, 5, 6],
        [7, 5, 6, 4]])

## Define the model

In [5]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size=64):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embed(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

model = CharRNN(vocab_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

## Training

In [6]:
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    output = model(X)
    loss = loss_fn(output, y)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 50, Loss: 0.0005
Epoch 100, Loss: 0.0003
Epoch 150, Loss: 0.0002
Epoch 200, Loss: 0.0002


## Predict next letter


In [16]:
def predict_next_letter(seed):
    model.eval()
    result = seed
    input_seq = [char2idx[c] for c in seed[-seq_length:]]
    x = torch.tensor([input_seq], dtype=torch.long)
    with torch.no_grad():
            output = model(x)
            next_idx = torch.argmax(output, dim=1).item()
            next_char = idx2char[next_idx]
            result += next_char
            seed += next_char
    return result

In [24]:
predict_next_letter('hell')

'hello'

## Text generation

In [41]:
def generate_text(seed, length=len(text)):
    model.eval()
    result = seed
    length -= len(seed)
    for _ in range(length):
        input_seq = [char2idx[c] for c in seed[-seq_length:]]
        x = torch.tensor([input_seq], dtype=torch.long)
        with torch.no_grad():
            output = model(x)
            next_idx = torch.argmax(output, dim=1).item()
            next_char = idx2char[next_idx]
            result += next_char
            seed += next_char
    return result

In [45]:
generate_text("hell")

'hello world'